In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing
from torchvision import datasets, transforms

In [2]:
torch.manual_seed(2018)

In [3]:
usd_cuda = torch.cuda.is_available()
device = torch.device('cuda')
kwargs = {'num_workers':4, 'pin_memory':True}

In [4]:
trans = transforms.Compose([
    #transforms.Scale(32), # 32by32
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train = True, download = True,
                transform = trans),
    batch_size = 256,
    shuffle=True,
    **kwargs)

Files already downloaded and verified


In [6]:
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train = False, download = False,
                transform = trans),
    batch_size = 256,
    shuffle=True,
    **kwargs)

In [7]:
#model
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,10,kernel_size=3)
        self.conv2 = nn.Conv2d(10,20, kernel_size=3)
        self.conv3 = nn.Conv2d(20,40, kernel_size=3)
#         self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1440,840)
        self.fc2 = nn.Linear(840,160)
        self.fc3 = nn.Linear(160,10)

    def forward(self, x):
        x = F.relu(self.conv1(x),2)
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        
        x = x.view(-1,1440)
        
        x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [8]:
model = Net().to(device)

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         if batch_idx % 1000:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'. format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100.*batch_idx / len(train_loader), loss.item()))

In [11]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [12]:
for epoch in range(1,31):
    train(epoch)
    test()
else:
    torch.save(model, 'main1.pt')


Test set: average loss: 1.4620, Accuracy: 4700/10000 (47%)


Test set: average loss: 1.2031, Accuracy: 5689/10000 (57%)


Test set: average loss: 1.0733, Accuracy: 6179/10000 (62%)


Test set: average loss: 0.9831, Accuracy: 6496/10000 (65%)


Test set: average loss: 0.9368, Accuracy: 6701/10000 (67%)


Test set: average loss: 0.8936, Accuracy: 6852/10000 (69%)


Test set: average loss: 0.9226, Accuracy: 6764/10000 (68%)


Test set: average loss: 0.8598, Accuracy: 7067/10000 (71%)


Test set: average loss: 0.8697, Accuracy: 7122/10000 (71%)


Test set: average loss: 0.9683, Accuracy: 7041/10000 (70%)


Test set: average loss: 0.9155, Accuracy: 7141/10000 (71%)


Test set: average loss: 1.0397, Accuracy: 7078/10000 (71%)


Test set: average loss: 1.1277, Accuracy: 7131/10000 (71%)


Test set: average loss: 1.2336, Accuracy: 7142/10000 (71%)


Test set: average loss: 1.3839, Accuracy: 7073/10000 (71%)


Test set: average loss: 1.5855, Accuracy: 6978/10000 (70%)


Test set: average loss:

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


<h3>Kaggle data</h3>

In [16]:
image_data = datasets.ImageFolder('./data/test/', transform=trans)
kaggle_loader = torch.utils.data.DataLoader(image_data,
                                            batch_size=256,
                                            shuffle = False,
                                           num_workers = 8)

In [27]:
CLASSES = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [28]:
model.eval()
for data, target in kaggle_loader:
    data = data.to(device)
    output = model(data)
    pred = output.max(1, keepdim = True)[1]
    print(CLASSES[pred])
    break

TypeError: only integer tensors of a single element can be converted to an index